In [36]:
from math import radians, cos, sin, asin, sqrt

EARTH_RADIUS = 6_371 

def haversine_distance(lat1 : float , lon1 : float, lat2 : float, lon2 : float, unit : str = "m") -> float:
    R = EARTH_RADIUS if unit == "km" else EARTH_RADIUS * 1_000
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R * c

In [37]:
import json 
class DeliveryOrder:
    def __init__(self, id : str, origin_lat : float, origin_long : float, dest_lat : float, dest_long : float, weight : int) -> None:
        self.id : str = id
        self.weight : int = weight
        self.start_position : dict = {
            "latitude": origin_lat,
            "longitude": origin_long
        }
        self.destination_position : dict = {
            "latitude": dest_lat,
            "longitude": dest_long
        }
        self.order_status : bool = False
        
    def __repr__(self) -> str:
        return json.dumps({
            "id": self.id,
            "dest_lat": round(self.destination_position['latitude'],2),
            "dest_long": round(self.destination_position['longitude'],2),
            "weight": self.weight
        })

In [38]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        for _ in range(1000):
            func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {(end_time - start_time):.6f} seconds to run 1000 times.")
    return wrapper

In [39]:
# ----------------------------------------------------------------------------------------------

from itertools import combinations

# ---------------------------------------------------------------------------------------------

def closest_order(latitude, longitude, orders : list[DeliveryOrder]) -> DeliveryOrder:
    min_dist = float('inf')
    closest = None
    for order in orders:
        dist = haversine_distance(
            latitude,
            longitude,
            order.destination_position['latitude'],
            order.destination_position['longitude']
        )
        if dist < min_dist:
            min_dist = dist
            closest = order
    return closest

# ---------------------------------------------------------------------------------------------

def generate_path(orders: list[DeliveryOrder], first_order: DeliveryOrder) -> list[DeliveryOrder]:
    if not orders:
        return []
    start_order = first_order
    if start_order not in orders:
        return []
    path = [start_order]
    visited = {start_order.id} 
    current_order = start_order
    while len(path) < len(orders):
        next_order = None
        min_distance = float('inf')
        for order in orders:
            if order.id not in visited:  
                distance = haversine_distance(
                    current_order.destination_position['latitude'], 
                    current_order.destination_position['longitude'], 
                    order.destination_position['latitude'], 
                    order.destination_position['longitude']
                )
                if distance < min_distance:
                    min_distance = distance
                    next_order = order
        if next_order:
            visited.add(next_order.id)  
            path.append(next_order)
            current_order = next_order
        else:
            break 
    return path

# ---------------------------------------------------------------------------------------------

def calculate_travel_distance(path : list[DeliveryOrder]) -> float:
    if len(path) < 2:
        return 0.0
    total_distance = 0.0
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        total_distance += haversine_distance(
            start.destination_position['latitude'], 
            start.destination_position['longitude'], 
            end.destination_position['latitude'], 
            end.destination_position['longitude']
        )
    return total_distance

# ---------------------------------------------------------------------------------------------

def calculate_capacity_level(orders: list[DeliveryOrder], max_capacity: int) -> float:
    total_weight = sum(order.weight for order in orders)
    capacity_level = total_weight / max_capacity
    return min(capacity_level, 1.0)

# ---------------------------------------------------------------------------------------------

def utility(travel_distance : float, velocity : float, capacity_level : float) -> float:
    travel_time = travel_distance / velocity
    time_k = 0.1
    capacity_k = 2.0
    if travel_time <= 0:
        return float('-inf')
    # decreases with increasing travel time, sharply penalizes near-zero travel time
    travel_utility = - time_k * (1 / travel_time + travel_time)
    # maximize at capacity_level == 1, penalize deviation from 1
    capacity_utility = - capacity_k * (1 - capacity_level)**2
    # TODO: penalizes solutions that do not have enough autonomy to recharge in X warehouses
    autonomy_utility = 0.0
    return travel_utility + capacity_utility + autonomy_utility

# ---------------------------------------------------------------------------------------------

def combine_orders(orders : list[DeliveryOrder], capacity : int) -> list[list[DeliveryOrder]]:
    all_combinations = []
    for r in range(1, len(orders) + 1):
        all_combinations.extend(combinations(orders, r))
    valid_combinations = []
    for combo in all_combinations:
        if sum(order.weight for order in combo) <= capacity:
            valid_combinations.append(list(combo))
    return valid_combinations

# ---------------------------------------------------------------------------------------------

def best_available_orders(orders: list[DeliveryOrder], latitude: float, longitude: float, capacity: int, velocity: float) -> list[DeliveryOrder]:
    order_sets = combine_orders(orders, capacity)
    best_set = None
    best_utility = float('-inf')
    for order_set in order_sets:
        first_order = closest_order(latitude, longitude, order_set)
        path = generate_path(order_set, first_order)
        travel_distance = calculate_travel_distance(path)
        capacity_level = calculate_capacity_level(order_set, capacity)
        set_utility = utility(travel_distance, velocity, capacity_level)
        print(f"Order Set: {order_set} - Utility: {set_utility} vs Best Utility: {best_utility}")
        if set_utility > best_utility:
            best_set = order_set
            best_utility = set_utility
    return best_set

# ----------------------------------------------------------------------------------------------

def best_order_decision(self) -> str:
    winner = None
    drone_utility = float('-inf')
    
    if self.next_orders:
        orders = self.next_orders
        closest = closest_order(self.position["latitude"], self.position["longitude"], orders)
        distance_closest_order = haversine_distance(
            self.position["latitude"], 
            self.position["longitude"], 
            closest.destination_position['latitude'], 
            closest.destination_position['longitude']
        )
        path = generate_path(orders, closest)
        travel_distance = distance_closest_order + calculate_travel_distance(path)
            
        capacity_level = calculate_capacity_level(orders, self.params.max_capacity)
        drone_utility = utility(travel_distance, self.params.velocity, capacity_level)
        
    # print available_order_sets
    print(f"{self.params.id} - [BEST ORDER DECISION] - {self.available_order_sets}")
    
    for warehouse, orders in self.available_order_sets.items():
        if self.next_orders:
            orders += self.next_orders
        distance_warehouse = haversine_distance(
            self.position["latitude"], 
            self.position["longitude"], 
            self.warehouse_positions[warehouse]['latitude'], 
            self.warehouse_positions[warehouse]['longitude']
        )
        closest_to_warehouse = closest_order(
            self.warehouse_positions[warehouse]['latitude'], 
            self.warehouse_positions[warehouse]['longitude'], 
            orders
        )
        distance_warehouse_to_closest_order = haversine_distance(
            self.warehouse_positions[warehouse]['latitude'], 
            self.warehouse_positions[warehouse]['longitude'], 
            closest_to_warehouse.destination_position['latitude'], 
            closest_to_warehouse.destination_position['longitude']
        )
        path = generate_path(orders, closest_to_warehouse)
        travel_distance = distance_warehouse + distance_warehouse_to_closest_order + calculate_travel_distance(path)
            
        orders += self.next_orders
        capacity_level = calculate_capacity_level(orders, self.params.max_capacity)
        new_utility = utility(travel_distance, self.params.velocity, capacity_level)
            
        if new_utility > drone_utility:
            winner = warehouse
            drone_utility = new_utility  
    
    return winner

In [40]:
orders_w1 : list[DeliveryOrder] = [
    DeliveryOrder("w_1", 1, 1, 0.1, 0.1, 5),
    DeliveryOrder("w_2", 1, 1, 0.2, 0.8, 5),
    DeliveryOrder("w_3", 1, 1, 0.3, 0.7, 5),
    DeliveryOrder("w_4", 1, 1, 0.4, 0.6, 5),
    DeliveryOrder("w_5", 1, 1, 0.5, 0.4, 5),
    DeliveryOrder("w_6", 1, 1, 0.6, 0.6, 5),
    DeliveryOrder("w_7", 1, 1, 0.7, 0.3, 5),
    DeliveryOrder("w_8", 1, 1, 0.8, 0.4, 5),
    DeliveryOrder("w_9", 1, 1, 0.9, 0.5, 5),
    DeliveryOrder("w_10",1, 1, 0.0, 0.3, 5),
]

orders_w2 : list[DeliveryOrder] = [
    DeliveryOrder("a_1", 1, 1, 0.1, 0.1, 5),
    DeliveryOrder("a_2", 1, 1, 0.2, 0.8, 5),
    DeliveryOrder("a_3", 1, 1, 0.3, 0.7, 5),
    DeliveryOrder("a_4", 1, 1, 0.4, 0.6, 5),
    DeliveryOrder("a_5", 1, 1, 0.5, 0.4, 5),
    DeliveryOrder("a_6", 1, 1, 0.6, 0.6, 5),
    DeliveryOrder("a_7", 1, 1, 0.7, 0.3, 5),
    DeliveryOrder("a_8", 1, 1, 0.8, 0.4, 5),
    DeliveryOrder("a_9", 1, 1, 0.9, 0.5, 5),
    DeliveryOrder("a_10",1, 1, 0.0, 0.3, 5),
]


In [41]:
w1 = best_available_orders(orders_w1, 0.4, 0.1, 20, 10)
w2 = best_available_orders(orders_w2, 0.4, 0.1, 20, 10)

print([order for order in w1])
print([order for order in w2])

available_sets = {
    "w1": w1,
    "w2": w2
}


Order Set: [{"id": "w_1", "dest_lat": 0.1, "dest_long": 0.1, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_2", "dest_lat": 0.2, "dest_long": 0.8, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_3", "dest_lat": 0.3, "dest_long": 0.7, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_4", "dest_lat": 0.4, "dest_long": 0.6, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_5", "dest_lat": 0.5, "dest_long": 0.4, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_6", "dest_lat": 0.6, "dest_long": 0.6, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_7", "dest_lat": 0.7, "dest_long": 0.3, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_8", "dest_lat": 0.8, "dest_long": 0.4, "weight": 5}] - Utility: -inf vs Best Utility: -inf
Order Set: [{"id": "w_9", "dest_lat": 0.9, "dest_long": 0.5, "weight": 5}] - Utility: -inf vs Be